In [9]:
from riemann import tx as rtx
from riemann import utils
from datetime import datetime, timezone
import pandas as pd
# from sqlalchemy import create_engine
# from sqlalchemy import Column, Integer, Text, MetaData, Table, String

# engine = create_engine('sqlite://')
# df = pd.DataFrame(columns=['txid', 'block_height', 'input_num', 'output_num', 'output_val', 'witness_idx', 'witness_data'])
# witness_data = Table(
#     Column('txid', String)
#     Column
# )

# Can specify m-of-n to get specific multisig type
# Use bytestrings for m-of-n b'\x52'
def get_multisig_witnesses(blocks, m=None, n=None):
    witness_data = []
    for block in blocks:
        block_height = block.height
        block_unix_time = block.header.timestamp.replace(tzinfo=timezone.utc).timestamp()

        for tx in block.transactions:
            if not tx.is_segwit:
                continue

            try:
                dtx = rtx.Tx.from_bytes(tx.hex)     
            except:
                print(f'Transaction errored. Txid: {tx.txid} Hex: {tx.hex}')   
                continue

            # LN multisigs have Segwit version 00 at the start of the stack
            if dtx and len(dtx.tx_witnesses[0].stack) > 0 and (dtx.tx_witnesses[0].stack[0] != b'\x00'):
                continue    

            txid = tx.txid
            input_num = tx.n_inputs
            output_num = tx.n_outputs

            # FIX: This is the total value of the transaction but should be the value of each individual output
            output_val = sum(utils.le2i(out.value) for out in dtx.tx_outs)

            for witness_idx, witness in enumerate(dtx.tx_witnesses):
                if len(witness.stack) == 0:
                    continue

                witness_script = witness.stack[-1].to_bytes()

                # Make sure script ends with op_checkmultisig                
                if witness_script[-1:] != b'\xAE':
                    continue 

                multisig_m = witness_script[1:2]
                if m and multisig_m != m:
                    continue
                    
                multisig_n = witness_script[-2:-1]
                if n and multisig_n != n:
                    continue
                
#                 outpoint_txid = dtx.tx_ins[witness_idx].outpoint.tx_id
#                 input_val = 
                for data in witness.stack:
                    witness_data.append({'block_unix_time' : block_unix_time,
                               'txid' : txid,
                               'block_height' : block_height,
                               'input_num' : input_num,
#                                'input_val' : input_val,
                               'output_num' : output_num,
                               'output_val' : output_val,
                               'witness_idx' : witness_idx, 
                               'witness_data' : data.hex()})

    return witness_data

In [7]:
# For making csvs
import os
import csv
from blockchain_parser.blockchain import Blockchain

blockchain = Blockchain(os.path.expanduser('/mnt/volume_nyc1_03/blocks/blocks/'))

start = 507000
end = 507300
with open(f'multisig_witnesses_{start}_{end}.csv', 'w', buffering=1, newline='') as csvfile:
    fieldnames = ['block_unix_time', 'txid', 'block_height', 'input_num', 'output_num', 'output_val', 'witness_idx', 'witness_data']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    step = 100
    for i in range(start, end, step):
        print(f'Getting blocks {i} to {i+step-1}')
        blocks = blockchain.get_ordered_blocks(os.path.expanduser('~/.bitcoin/blocks/index'), start=i, end=i+step-1)
        witness_data = get_multisig_witnesses(blocks, m=b'\x52', n=b'\x52')
        
        print(f'Writing blocks {i} to {i+step-1}')
        for data in witness_data:
            writer.writerow(data)

Getting blocks 507000 to 507099
Writing blocks 507000 to 507099
Getting blocks 507100 to 507199
Writing blocks 507100 to 507199
Getting blocks 507200 to 507299
Writing blocks 507200 to 507299


In [ ]:
# For testing
import os
from blockchain_parser.blockchain import Blockchain

blockchain = Blockchain(os.path.expanduser('/mnt/volume_nyc1_03/blocks/blocks/'))
start = 507000
end = 507001

blocks = blockchain.get_ordered_blocks(os.path.expanduser('~/.bitcoin/blocks/index'), start=start, end=end, cache=f'index-cache_{start}_{end}.pickle')
result = get_multisig_witnesses(blocks, m=b'\x52', n=b'\x52')

display(result)